# Đồ án cuối kỳ NMKHDL

Câu hỏi cần trả lời

## Phần 1: Thu thập dữ liệu

In [2]:
import requests
from bs4 import BeautifulSoup

### Get tất cả các link laptop

In [29]:
all_brands_links = [
    'https://phongvu.vn/laptop-asus-1058.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-acer-1059.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-Dell-1056.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/macbook-1061.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-hp-1057.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-lenovo-1060.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-lg-1268.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=',
    'https://phongvu.vn/laptop-msi-1081.cat?pv_source=homepage&pv_medium=de-megamenu-text&page='
]

items = [] # danh sách các link laptop

# parse để lấy link
for brand in all_brands_links:
    for page in range(1, 50):
        pv_laptop_url = brand + str(page)
        while True:
            try:
                html_text = requests.get(pv_laptop_url).text
                break
            except:
                continue
        tree = BeautifulSoup(html_text, 'html.parser')
        found = tree.find_all('a', {'class': 'css-1rhapru'})
        if len(found) == 0:
            break
        items = items + found
    print(brand)

https://phongvu.vn/laptop-asus-1058.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=
https://phongvu.vn/laptop-acer-1059.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=
https://phongvu.vn/laptop-Dell-1056.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=
https://phongvu.vn/macbook-1061.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=
https://phongvu.vn/laptop-hp-1057.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=
https://phongvu.vn/laptop-lenovo-1060.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=
https://phongvu.vn/laptop-lg-1268.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=
https://phongvu.vn/laptop-msi-1081.cat?pv_source=homepage&pv_medium=de-megamenu-text&page=


In [30]:
len(items)

794

Tổng cộng get được 794 link laptop.

In [31]:
# xem qua các link
for item in items[:5]:
    print(item['href'])

/may-tinh-xach-tay-laptop-asus-d509da-ej800t-amd-ryzen-3-3250u-bac-s200900077.html
/may-tinh-xach-tay-laptop-asus-vivobook-flip-tm420ia-ec155-amd-ryzen-3-4300u-xam-s201104484.html
/may-tinh-xach-tay-laptop-asus-tuf-gaming-fa506iial012t-amd-ryzen-5-4600h-xam-s200500682.html
/may-tinh-xach-tay-laptop-asus-vivobook-x509ja-ej427t-i3-1005g1-bac-s201100788.html
/may-tinh-xach-tay-laptop-asus-tuf-gaming-fx506li-hn039t-i5-10300h-xam-s201100021.html


In [34]:
import csv

### Parse từng link và ghi ra file csv

File dữ liệu: `data.csv`

In [35]:
fields = {'Thương hiệu': 'Brand', 'Bảo hành': 'Warranty', 'Màu sắc': 'Color', 'Series laptop': 'SeriesLaptop', 'Part-number': 'PartNum', 'Thế hệ CPU': 'CPUgen', 'CPU': 'CPU', 'Chip đồ họa': 'GraphicChip', 'RAM': 'RAM', 'Màn hình': 'Screen', 'Lưu trữ': 'Storage', 'Số cổng lưu trữ tối đa': 'MaxStoPortNum', 'Kiểu khe M.2 hỗ trợ': 'SupportM2', 'Cổng xuất hình': 'OutVideoPort', 'Cổng kết nối': 'ConnectPort', 'Kết nối không dây': 'Wireless', 'Bàn phím': 'Keyboard', 'Hệ điều hành': 'OS', 'Kích thước': 'Size', 'Pin': 'Pin', 'Khối lượng': 'Weight', 'Đèn LED trên máy': 'LED', 'Phụ kiện đi kèm': 'Accessories', 'Bảo mật': 'Security', 'Ổ đĩa quang': 'OptDrive', 'Tính năng': 'Feature'}

fieldnames = ['SKU', 'Title', 'Price', 'Brand', 'Warranty', 'Color', 'SeriesLaptop', 'PartNum', 'CPUgen', 'CPU', 'GraphicChip', 'RAM', 'Screen', 'Storage', 'MaxStoPortNum', 'SupportM2', 'OutVideoPort', 'ConnectPort', 'Wireless', 'Keyboard', 'OS', 'Size', 'Pin', 'Weight', 'LED', 'Accessories', 'Security', 'OptDrive', 'Feature']

In [36]:
file = open('data.csv', 'w', encoding='utf-8')
file_writer = csv.DictWriter(file, fieldnames=fieldnames)
file_writer.writeheader()

count = 0
for item in items:
    count += 1
    url = 'https://phongvu.vn' + item['href']
    while True:
        try:
            laptop_html_text = requests.get(url).text
            break
        except:
            continue
    laptop_html_tree = BeautifulSoup(laptop_html_text, 'html.parser')
    title = laptop_html_tree.find('div', {'class': 'css-1jpdzyd'}).text
    sku = laptop_html_tree.find('div', {'class': 'css-5nimvs'}).text.split(' ')[-1]
    price = laptop_html_tree.find('span', {'class': 'css-3725in'}).text[:-1]
    values = {'SKU': sku, 'Title': title, 'Price': price} 
    for i in laptop_html_tree.find_all('div', {'class': 'css-7j9rw7'}):
        name = i.find('span', {'class': 'css-6z2lgz'}).text.strip(' ')
        if name in fields:
            values[fields[name]] = i.find('div', {'class': 'css-111s35w'}).text.strip(' ')
    file_writer.writerow(values)
    if count % 10 == 0:
        print(count)
file.close()

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790


## Phần 2: Khám phá dữ liệu

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

In [9]:
data_df = pd.read_csv('data.csv', index_col='SKU') # Dùng mã SKU của sản phẩm để làm index
data_df.head()

,Title,Price,Brand,Warranty,Color,SeriesLaptop,PartNum,CPUgen,CPU,GraphicChip,...,Keyboard,OS,Size,Pin,Weight,LED,Accessories,Security,OptDrive,Feature
SKU,,,,,,,,,,,,,,,,,,,,,
200900077,"Laptop ASUS D509DA EJ800T ( 15.6"" Full HD/AMD ...",10.190.000,ASUS,24,Bạc,NaN,EJ800T,"Ryzen 3 , AMD Ryzen thế hệ thứ 3",AMD Ryzen 3 3250U ( 2.6 GHz - 3.5 GHz / 4MB / ...,AMD Radeon Graphics,...,"thường , có phím số , không đèn",Windows 10 Home 64-bit,36 x 23.5 x 2.29 cm,2 cell Pin liền,1.9 kg,không đèn,NaN,NaN,NaN,NaN
201104484,"Laptop ASUS Vivobook Flip TM420IA EC155 ( 14"" ...",13.490.000,ASUS,24,Xám,VivoBook,EC155,"Ryzen 3 , AMD Ryzen thế hệ thứ 3",AMD Ryzen 3 4300U ( 2.7 GHz - 3.7 GHz / 4MB / ...,AMD Radeon Graphics,...,"thường , không phím số , LED",Windows 10 Home 64-bit,32.4 x 22 x 1.82 cm,"3 cell 42 Wh , Pin liền",1.5 kg,không đèn,NaN,NaN,NaN,Numberpad ảo tích hợp trên touchpad
200500682,"Laptop ASUS TUF Gaming FA506II AL012T ( 15.6"" ...",21.990.000,ASUS,24,Xám,TUF Gaming,AL012T,"Ryzen 5 , AMD Ryzen thế hệ thứ 4",AMD Ryzen 5 4600H ( 3.0 GHz - 4.0 GHz / 8MB / ...,NVIDIA GeForce GTX 1650Ti 4GB GDDR6 / AMD Ra...,...,"thường , có phím số , RGB",Windows 10 Home SL 64-bit,35.98 x 25.6 x 2.28 cm,"3 cell 48 Wh , Pin liền",2 kg,không đèn,NaN,NaN,NaN,NaN
201100788,"Laptop ASUS Vivobook X509JA EJ427T ( 15.6"" Ful...",11.190.000,ASUS,24,Bạc,VivoBook,EJ427T,"Core i3 , Intel Core thế hệ thứ 10",Intel Core i3-1005G1 ( 1.2 GHz - 3.4 GHz / 4MB...,Intel UHD Graphics,...,"thường , có phím số , không đèn",Windows 10 Home SL 64-bit,36.02 x 23.49 x 2.29 cm,"2 cell 32 Wh , Pin liền",1.7 kg,không đèn,NaN,Vân tay,NaN,NaN
201100021,"Laptop ASUS TUF Gaming FX506LI HN039T ( 15.6"" ...",23.490.000,ASUS,24,Xám,TUF Gaming,HN039T,"Core i5 , Intel Celeron 1000",Intel Core i5-10300H ( 2.5 GHz - 4.5 GHz / 8MB...,NVIDIA GeForce GTX 1650Ti 4GB GDDR6 / Intel ...,...,"thường , có phím số , RGB",Windows 10 Home 64-bit,35.9 x 25.6 x 2.47 cm,"3 cell 48 Wh , Pin liền",2.3 kg,không đèn,"Adapter, Sticker, Sách HDSD, Sổ BH",NaN,NaN,NaN


### Ý nghĩa các cột

In [ ]:
with open('description.txt', 'r') as f:
    print(f.read())

In [10]:
data_df.shape

(794, 28)

In [11]:
data_df.index.duplicated().sum()

0

Không có dòng trùng.

In [8]:
# Cột dùng làm output là Price
data_df['Price'].dtype

dtype('O')

Cột output có dữ liệu dạng Object nên chúng ta cần chuyển về dạng số để phù hợp với bài toán hồi quy.

In [5]:
data_df['Price'].isna().sum()

0

Cột output không có giá trị thiếu

## Phần 3: Tiền xử lý (tách các tập)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn import set_config
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

## Phần 4: Khám phá dữ liệu (Tập huấn luyện)

### Mỗi cột input hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [ ]:
train_X_df.dtypes

### Với mỗi cột input có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

## Phần 5: Tiền xử lý (tập huấn luyện)

Thêm và bớt các cột 

In [ ]:
def convert_col_dtype(col):
    if col.name == 'SSD':
        col.replace('1','1000', inplace = True)
        col.replace('2','2000', inplace = True)

        return pd.to_numeric(col, errors='coerce')
    if col.name == 'gen':
        col.replace('1000',np.NaN,inplace = True)
        return pd.to_numeric(col, errors='coerce')
    if col.name == 'Security':     
        return col.apply(lambda x: 'Yes' if not pd.isnull(x) else 'No')
    if col.name in ['Pin','Weight','Screen','RAM']:  
        return pd.to_numeric(col, errors='coerce')  
    if col.name == 'Price':
        col = col.str.replace('.', '')
        return pd.to_numeric(col, errors='coerce')
    return col

## Phần 6: Tiền xử lý (tập validation)

## Phần 7: Tiền xử lý + mô hình hóa

### Tìm mô hình tốt nhất

### Đánh giá mô hình tìm được